In [1]:
import numpy as np
import math
from ipywidgets import *
import matplotlib.pyplot as plt

vel_rel = np.linspace(0.0, 1000., 100) #relative velocity between gas and body in m/s
dens = 3000. #lunar density in kg/m3; 3.0 g/cc

def dragfunc(rad_body, C_D, dens_gas):
    #mass of body in kg; assume spherical body (and spherical cross section)
    mass_body = dens*(4./3.)*math.pi*(rad_body**3)
    #acceleration imparted on body due to gas drag in m/s^2
    a_drag = 0.5*C_D*math.pi*(rad_body**2)*dens_gas*(vel_rel**2)/mass_body
    plt.plot(vel_rel, a_drag)
    plt.xlabel('velocity of the gas relative to the velocity of the moonlet (m/s)')
    plt.ylabel('gas drag acceleration (m/s$^2$)')
    plt.show()
    plt.close()
   
#RELATIVE SCALE IN GENERAL: how big the moonlets are (marble, golf ball, grain of sand,
#yoga ball, soccer field, etc.), how fast the relative and absolute velocity is   
style = {'description_width': 'initial'}
layout = {'width': '400px'}
%matplotlib inline
interact(dragfunc,
         rad_body=FloatLogSlider(value=1, base=10, min=-3, max=6, step=1,
                                 description='Size of moonlet (m)', style=style, layout=layout),
         C_D=FloatSlider(value=2, min=0, max=2.5, step=0.5, description='Gas drag coefficient C$_D$ (-)',
                         continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         dens_gas=FloatLogSlider(value=1, base=10, min=-3, max=3, step=1,
                                 description='Gas density (kg/m$^3$)', style=style, layout=layout)
        )

#acceleration in terms of what we experience on Earth
accel_label=['on board space shuttle ISS','elevator headed up','gravitational pull on surface of Earth',
             'roller coaster launch','space shuttle launch, gravitron amusement ride','top of roller coaster loop',
             'maximum heavy braking on Formula One Car','maximum acceleration permitted on piloted planes',
             'hard slap to the face, car crash','shock capacity of mechanical wrist watch',
             'Mantis Shrimp claw strike','proton in Large Hadron Collider']
accel_g=['1e-6 g','0.2 g','1 g','1.5 g','3 g','5 g','6.3 g','10 g','100 g','5000 g','10400 g','1.9e8 g']
left_box0=widgets.VBox([widgets.Label(str(i)) for i in accel_label])
right_box0=widgets.VBox([widgets.Label(str(i)) for i in accel_g])

#velocity in terms of what we experience on Earth
vel_label=['Walking','Biking','Usain Bolt sprinting','Train','Car moving 100 mph (160 km/h)',
           'Helicopter','Commerical airplane','Speed of sound in air (sound barrier)']
vel_ms=['1.5 m/s','4.5 m/s','12.5 m/s','27 m/s','45 m/s','70 m/s','220 m/s','343 m/s']
left_box1=widgets.VBox([widgets.Label(str(i)) for i in vel_label])
right_box1=widgets.VBox([widgets.Label(str(i)) for i in vel_ms])

widgets.HBox([left_box0,right_box0,left_box1,right_box1])

interactive(children=(FloatLogSlider(value=1.0, description='Size of moonlet (m)', layout=Layout(width='400px'…

In [2]:
#Dear Reader: If using CoLab, remove the # from the line below and shift-return to install rebound
#!pip install rebound

In [3]:
# interactive feature
import math
import numpy as np
import rebound
import matplotlib.pyplot as plt
from IPython.display import clear_output

Mass_syn=5e24 #Earth mass synestia in kg
dens=3000. #density of moonlet kg/m3
G_hr=(6.674e-11)*(3600**2) #gravitational constant converted to m^3 kg^-1 hr^-2
const1=10.55#10.55#10 to 11
const2=0.86#0.86#0.85 to 0.9
const3=0.9e35#0.9e35 #1e33 to 1e37
const4=-4.7#-4.7 #-4.5 to -5

def pltdrag(r0,x0,y0,z0):
    clear_output(wait=True)
    f = FloatProgress(min=0, max=100) # instantiate the bar
    display(f) # display the bar
    m0=4.*math.pi*dens*(r0**3)/3. #mass of moonlet in kg
    s0=(x0**2 + y0**2)**0.5 #initial moonlet distance from rotational axis in m
    v_gas=0.95*(G_hr*Mass_syn/s0)**0.5 #sub-Keplerian gas velocity m/hr #0.01*(10.**(0.15599001870969695*np.log10(s0) - 1.6401471197424731))*((G_hr*Mass_syn/s0)**0.5)
    vx0=-v_gas*y0/s0 #x-component of sub-Keplerian gas velocity m/hr
    vy0=v_gas*x0/s0 #y-component of sub-Keplerian gas velocity m/hr

    #intialize orbit simulation
    sim_drag = rebound.Simulation() #start simulation
    sim_drag.units = ('Hr', 'M', 'Kg') #use SI units, seconds not available
    sim_drag.add(m=Mass_syn) #add central body: Earth mass synestia
    sim_drag.add(m=m0,r=r0) #add moonlet to simulation
    ps_drag = sim_drag.particles #initialize particle class (holds xyz position information)
    ps_drag[1].x=x0 #set initial x position of moonlet in m
    ps_drag[1].y=y0 #set initial y position of moonlet in m
    ps_drag[1].z=z0 #set initial z position of moonlet in m
    ps_drag[1].vx=vx0 #set initial x velocity of moonlet in m/hr #intial velocities will be same as gas
    ps_drag[1].vy=vy0 #set initial y velocity of moonlet in m/hr
    ps_drag[1].vz=0 #set initial z velocity of moonlet in m/hr #assume gas is not moving in z
    sim_drag.integrator = "ias15"#"whfast"#"leapfrog"#symplectic integrator #"ias15" #default integrator; adaptive time-stepping
    sim_drag.min_dt=0.001 #make sure doesn't pick too small of a time step
    sim_drag.dt=0.001 #start with this time step
    f.value += 5

    #define gas drag acceleration
    def drag(sim_drag):
        #drag coefficient is 2 in potential Moon-forming synestias
        ps_x = ps_drag[1].x
        ps_y = ps_drag[1].y
        ps_z = ps_drag[1].z
        ps_vx = ps_drag[1].vx
        ps_vy = ps_drag[1].vy
        ps_vz = ps_drag[1].vz
        s=np.sqrt(ps_x**2 + ps_y**2) #cylindrical radius of moonlet in m
        z_s=const1*(s**const2) #scale height fit in m
        rho_g=const3*(s**const4)*np.exp(-(ps_z/z_s)**2) #gas density fit in kg/m^3
        v_g=0.95*np.sqrt(G_hr*Mass_syn/s) #sub-Keplerian gas velocity in m/hr #(10.**(0.15599001870969695*np.log10(s) - 1.6401471197424731))*((reb_sim.G*Mass_syn/s)**0.5)
        theta=np.arctan(ps_y/ps_x)
        v_gx=-np.abs(v_g*np.sin(theta))*np.sign(ps_y) #x component of gas velocity in m/hr
        v_gy=np.abs(v_g*np.cos(theta))*np.sign(ps_x) #y component of gas velocity in m/hr
        #assumption: speed of gas in z, v_gz, will be zero for synestia in hydrostatic equilibrium
        v_rel_x=ps_vx - v_gx
        v_rel_y=ps_vy - v_gy
        v_rel=np.sqrt(v_rel_x**2 + v_rel_y**2 + ps_vz**2)
        factor = math.pi*(ps_drag[1].r**2)*rho_g*v_rel/ps_drag[1].m #common factor for gas drag in components
        ps_drag[1].ax += -factor*v_rel_x
        ps_drag[1].ay += -factor*v_rel_y
        ps_drag[1].az += -factor*ps_vz

    #tell simulation to add gas drag and update velocities with each time step
    sim_drag.additional_forces = drag
    sim_drag.force_is_velocity_dependent = 1
    sim_drag.move_to_com() #move to center of mass (central body is at origin)

    Noutputs = 2000 #increase this to get a smoother curve for orbit
    if r0 < 10:
        times = np.linspace(0.,600., Noutputs)
    elif r0 > 1e4:
        times = np.linspace(0.,600., Noutputs)
    else:
        times = np.linspace(0.,200., Noutputs) #times to output position information of particle

    #intialize holder arrays for positional output from simulation
    xholder = np.zeros(Noutputs)
    yholder = np.zeros(Noutputs)
    zholder = np.zeros(Noutputs)
    sholder = np.zeros(Noutputs)
    f.value += 5
    fstep = 81./len(times)
    for k,time in enumerate(times):
        sim_drag.integrate(time)
        xtemp = ps_drag[1].x
        ytemp = ps_drag[1].y
        stemp = np.sqrt(xtemp**2 + ytemp**2)
        f.value += fstep
        if stemp <= 10e6:
            #exit out of simulation when reach the inner region (10,000,000 m)
            #shorten arrays to exclude zeros at end
            xholder = xholder[:k]
            yholder = yholder[:k]
            zholder = zholder[:k]
            sholder = sholder[:k]
            times = times[:k]
            f.value = 91
            print('The length of time the moonlet spends in the disk-like region is {0:.1f} hr.'.format(times[-1]))
            break
        else:
            xholder[k] = xtemp  # This stores the data which allows us to plot it later
            yholder[k] = ytemp
            zholder[k] = ps_drag[1].z
            sholder[k] = stemp
    if sholder[-1] > 11e6:
        print('The moonlet remains in the disk-like region at {0:.1f} hr.'.format(times[-1]))
    xholder = xholder*(1e-3)
    yholder = yholder*(1e-3)
    zholder = zholder*(1e-3)
    sholder = sholder*(1e-3)
    rholder = np.sqrt(sholder**2 + zholder**2)
    ymax=np.amax(rholder)

    #plot
    fig = plt.figure(figsize=(18,4))
    ax = plt.subplot(131)
    plt.plot(xholder, yholder)
    plt.plot(x0/1e3,y0/1e3,'x',markersize=10)
    plt.plot(0,0,'k*')
    plt.plot(xholder[-1], yholder[-1],'o')
    plt.xlabel('x (km)')
    plt.ylabel('y (km)')
    plt.ylim([-ymax,ymax])
    plt.xlim([-ymax,ymax])
    ax.set_aspect(1)
    
    f.value += 3

    ax2 = plt.subplot(132)
    plt.plot(sholder, zholder)
    plt.plot(s0/1e3,z0/1e3,'x',markersize=10)
    plt.plot(0,0,'k*')
    plt.plot(sholder[-1], zholder[-1],'o')
    plt.xlabel('distance to rotational axis r$_{xy}$ (km)')
    plt.ylabel('vertical distance off midplane z (km)')
    plt.ylim([-ymax,ymax])
    plt.xlim(xmin=0)
    ax2.axis('equal')

    f.value += 3
    
    ax3 = plt.subplot(133)
    plt.plot(times,rholder)
    plt.plot(0,rholder[0],'x',markersize=10)
    plt.plot(times[-1],rholder[-1],'o')
    plt.xlabel('time (hr)')
    plt.ylabel('radius r (km)')
    plt.show()
    plt.close()
    
    f.value += 3

    
x0slider = FloatSlider(value=20e6, min=0, max=60e6, step=20e6, description='x starting position of moon (m)',
                         continuous_update=False, readout=True, readout_format='.0f', style=style, layout=layout)
y0slider = FloatSlider(value=20e6, min=0, max=60e6, step=20e6, description='y starting position of moon (m)',
                         continuous_update=False, readout=True, readout_format='.0f', style=style, layout=layout)

def rxy_min_x0(*args):
    rxy0 = np.sqrt(x0slider.value**2 + y0slider.value**2)
    if x0slider.value < 20e6:
        y0slider.value = 20e6

def rxy_min_y0(*args):
    rxy0 = np.sqrt(x0slider.value**2 + y0slider.value**2)
    if y0slider.value < 20e6:
        x0slider.value = 20e6
    
x0slider.observe(rxy_min_x0, 'value')
y0slider.observe(rxy_min_y0, 'value')
interact_manual(pltdrag,
         r0=widgets.Dropdown(
             options=[('Coarse sand (mm)', 1.e-3),
                      ('Diameter of a belly button (cm)', 1.e-2),
                      ('Coconut (dm)', 0.1),
                      ('Cow (m)', 1.),
                      ('230 story building (km)', 1.e3),
                      ('Width of the United States (Mm)', 1.e6),
                      ('The Moon (3.5 Mm)', 3.5e6)],
             value=1.e3, description='Size (diameter) of moonlet:', continuous_update=False, style=style),
         x0=x0slider,
         y0=y0slider,
         z0=FloatSlider(value=0, min=0, max=30e6, step=5e6, description='z starting position of moon (m)',
                         continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
    )

interactive(children=(Dropdown(description='Size (diameter) of moonlet:', index=4, options=(('Coarse sand (mm)…

<function __main__.pltdrag(r0, x0, y0, z0)>